In [1]:
from pathlib import Path
import pandas as pd

In [2]:
folder_path = Path('data')
zip_files = list(folder_path.glob('*.zip'))
print(zip_files)
zip_file = zip_files[-1].absolute()
print(zip_file)

[WindowsPath('data/2023-04-26_00-54-lenta.zip'), WindowsPath('data/2023-04-27_14-27-lenta.zip'), WindowsPath('data/2023-04-30_13-43-lenta.zip'), WindowsPath('data/2023-05-07_21-14-lenta.zip'), WindowsPath('data/2023-05-13_20-33-lenta.zip')]
c:\works\work_002\data\2023-05-13_20-33-lenta.zip


In [3]:
df = pd.read_csv(zip_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30220 entries, 0 to 30219
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Категория      30220 non-null  object 
 1   Название       30220 non-null  object 
 2   Страна         30101 non-null  object 
 3   Скидка         15990 non-null  object 
 4   Цена           30220 non-null  float64
 5   Цена по карте  30218 non-null  float64
dtypes: float64(2), object(4)
memory usage: 1.4+ MB


In [4]:
mask = df['Название'].str.contains('Килька')
kilka = df[mask].copy()
mask2 = kilka['Категория'] == 'Бакалея'
mask3 = kilka['Название'].str.lower()#.contains('лента')
mask3 = mask3.str.contains('лента')
test = kilka[mask2 & mask3].sort_values(by='Цена').copy()
test.reset_index()

,index,Категория,Название,Страна,Скидка,Цена,Цена по карте
0,6042,Бакалея,"Килька ЛЕНТА в томатном соусе, 240г","Россия, 240 г",-31%,94.79,64.99


In [5]:
mask = df['Название'].str.lower().str.contains(r'(?=молоко)(?=.* пастериз)(?=.*без змж)', regex=True)
kilka = df[mask].copy()
mask2 = kilka['Категория'] == 'Бакалея'
mask3 = kilka['Название'].str.lower().str.contains('лента')
# test = kilka[mask2].sort_values(by='Цена').copy()
test = kilka.copy()
test['volume'] = test['Страна'].str.extract('(\d+)').astype(int)
test['Пр. цена'] = round((test['Цена по карте'] / (test['volume'] / 1000)), 2)
mask4 = test['volume']>600
test_min_mean = test[mask4]['Пр. цена'].nsmallest(5).mean()
test_median = test[mask4]['Пр. цена'].median()
test_max_mean = test[mask4]['Пр. цена'].nlargest(5).mean()
print(f'минимум \t{test_min_mean:.2f}')
print(f'медиана \t{test_median:.2f}')
print(f'максимум \t{test_max_mean:.2f}')
print(f'количество позиций: {test.shape[0]}')
display(test.nsmallest(5, columns='Пр. цена'))

минимум 	68.43
медиана 	83.56
максимум 	123.65
количество позиций: 39


,Категория,Название,Страна,Скидка,Цена,Цена по карте,volume,Пр. цена
1620,"Молоко, сыр, яйцо","Молоко пастеризованное СЧАСТЛИВОЕ ДЕТСТВО 1,5%...","Россия, 900 мл",-15%,64.79,54.99,900,61.10
1610,"Молоко, сыр, яйцо","Молоко пастеризованное ЛЕБЕДЯНЬМОЛОКО 2,5%, бе...","Россия, 900 г",-17%,70.49,57.99,900,64.43
3116,"Молоко, сыр, яйцо","Молоко пастеризованное БЕЖИН ЛУГ 2,5%, без змж...","Россия, 1400 г",-26%,136.89,99.99,1400,71.42
1622,"Молоко, сыр, яйцо","Молоко пастеризованное СЧАСТЛИВОЕ ДЕТСТВО 3,2%...","Россия, 900 мл",-27%,89.39,64.99,900,72.21
2540,"Молоко, сыр, яйцо",Молоко пастеризованное пастеризованный БЕЖИН Л...,"Россия, 1000 г",-36%,115.69,72.99,1000,72.99


In [6]:
display(test[test['volume']>600].nlargest(5, columns='Пр. цена'))

,Категория,Название,Страна,Скидка,Цена,Цена по карте,volume,Пр. цена
2802,"Молоко, сыр, яйцо",Молоко пастеризованное PARMALAT Comfort безлак...,"Россия, 900 мл",NaN,164.29,155.99,900,173.32
1968,"Молоко, сыр, яйцо",Молоко пастеризованное PARMALAT Comfort безлак...,"Россия, 900 мл",NaN,147.39,139.99,900,155.54
1706,"Молоко, сыр, яйцо","Молоко пастеризованное ПРОСТОКВАШИНО 3,2%, без...","Россия, 1400 мл",NaN,143.09,135.89,1400,97.06
1841,"Молоко, сыр, яйцо",Молоко пастеризованное ВКУСНОТЕЕВО ультрачисто...,"Россия, 900 г",NaN,91.59,86.99,900,96.66
1689,"Молоко, сыр, яйцо",Молоко пастеризованное ДОМИК В ДЕРЕВНЕ Деревен...,"Россия, 930 мл",-18%,109.49,88.99,930,95.69


In [7]:
import requests
from bs4 import BeautifulSoup as BS
import lxml
from pprint import pprint

url = 'https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D1%82%D1%80%D0%B5%D0%B1%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D1%81%D0%BA%D0%B0%D1%8F_%D0%BA%D0%BE%D1%80%D0%B7%D0%B8%D0%BD%D0%B0'

response = requests.get(url)
soup = BS(response.content, 'lxml')

table = soup.find('table', class_='wikitable')
rows = list(table.find_all('tr'))[2:]

sum_list = []
for row in rows[:]:
    out_list = []
    for index, element in enumerate(row.find_all('td')):
        if index > 1:
            per_year = float(element.get_text(strip=True).replace(',', '.'))
            per_month = per_year / 12
            out_list.append(round(per_month, 2))
        else:
            out_list.append(element.get_text(strip=True))
    sum_list.append(out_list)

table_df = pd.DataFrame(sum_list, columns=['name', 'unit', 'works', 'pensioners', 'children'])
table_df

,name,unit,works,pensioners,children
0,Хлебные продукты (хлебимакаронные изделияв пер...,кг,10.54,8.18,6.47
1,Картофель,кг,8.37,6.67,7.34
2,Овощиибахчевые,кг,9.55,8.17,9.38
3,Фруктысвежие,кг,5.00,3.75,9.84
4,Сахарикондитерские изделияв пересчете на сахар,кг,1.98,1.77,1.82
5,Мясопродукты,кг,4.88,4.50,3.67
6,Рыбопродукты,кг,1.54,1.33,1.55
7,Молокои молокопродукты в пересчете на молоко,кг,24.17,21.48,30.06
8,Яйца,штука,17.50,16.67,16.75
9,"Масло растительное,маргарини другиежиры",кг,0.92,0.83,0.42


In [79]:
mask = df['Название'].str.lower().str.contains(r'(?=.*стиральный.*)(?=.*порошок.*)', regex=True)
df_test = pd.DataFrame(df[mask].reset_index(drop='index'))
# mask2 = df_test['Название'].str.lower().str.contains(r'(?=.*)', regex=True)
df_test['Вес'] = df_test['Страна'].apply(lambda x: ''.join(x.split(',', maxsplit=1)[1:]))
df_test['Вес'] = df_test['Вес'].str.extract('(\d+\,?\d*)')
df_test['Вес'] = df_test['Вес'].str.replace(',', '.')
df_test['Вес'] = df_test['Вес'].astype(float).apply(lambda x: x if x > 100 else x * 1000).astype(int)

df_test['Страна'] = df_test['Страна'].apply(lambda x: x.split(',')[0])
# print(df_test['Вес'].unique())
df_test['За кг'] = df_test['Цена по карте'] / df_test['Вес'] * 1000
df_test['За кг'] = df_test['За кг'].round().astype(int)
# df_test[df_test['Страна'] == 'Дания']
df_test[~(df_test['Страна'] == 'Россия')].nsmallest(n=20, columns='За кг')

,Категория,Название,Страна,Скидка,Цена,Цена по карте,Вес,За кг
70,Бытовая химия,Стиральный порошок МАРА Узор Чысціні Утренняя ...,Беларусь,-31%,421.09,289.00,3000,96
47,Бытовая химия,"Стиральный порошок ABC Лимон, 3кг",Турция,-37%,557.89,349.00,3000,116
26,Бытовая химия,"Стиральный порошок ABC Горная свежесть, 3кг",Турция,-37%,557.89,349.00,3000,116
33,Бытовая химия,"Стиральный порошок для цветного белья ABC, 3кг",Турция,-37%,557.89,349.00,3000,116
41,Бытовая химия,"Стиральный порошок ABC Лаванда, 4кг",Турция,-36%,736.89,469.00,4000,117
44,Бытовая химия,"Стиральный порошок для цветного белья ABC, 4кг",Турция,-36%,736.89,469.00,4000,117
45,Бытовая химия,"Стиральный порошок ABC Горная свежесть, 4кг",Турция,-36%,736.89,469.00,4000,117
27,Бытовая химия,"Стиральный порошок для цветного белья ABC, 1,5кг",Турция,-38%,294.79,179.99,1500,120
122,Бытовая химия,"Стиральный порошок YETIS Горная свежесть, 6кг",Турция,NaN,768.49,729.99,6000,122
108,Бытовая химия,"Стиральный порошок YETIS Color, 4,5кг",Турция,NaN,578.99,549.99,4500,122
